In [1]:
import pandas as pd

comp_rec = pd.read_csv('./output/complete-record.csv')
soccerbase = pd.read_csv('./output/soccerbase.csv')
transfermarkt = pd.read_csv('./output/transfermarkt.csv')

In [2]:
def replace_special_chars(name):
    if 'Kané' not in name:
        name = name.replace('á', 'a')
        name = name.replace('é', 'e')
        name = name.replace('é', 'e')
        name = name.replace('è', 'e')
        name = name.replace('É', 'E')
        name = name.replace('ô', 'o')
    return name

dfs = [comp_rec, soccerbase, transfermarkt]
cols = ['player_id', 'surname', 'forename', 'player_name']

for df in dfs:
    for col in cols:
        df[col] = df[col].apply(replace_special_chars)

In [3]:
print(len(comp_rec))
print(len(soccerbase))
print(len(transfermarkt))

612
465
587


In [4]:
player_ids = pd.concat([comp_rec, soccerbase, transfermarkt])['player_id'].drop_duplicates().to_list()

len(player_ids)

1079

In [5]:
all_players = pd.merge(comp_rec, soccerbase, on=['player_id', 'surname', 'forename', 'player_name', 'player_dob'], how='outer').merge(transfermarkt, on=['player_id', 'surname', 'forename', 'player_name', 'player_dob'], how='outer').drop_duplicates().reset_index(drop=True)

print(len(all_players))

all_players.head(5)

1079


player_id     surname forename      player_name  player_dob  \
0      ACourtAlan19340930     A'Court     Alan     Alan A'Court  1934-09-30   
1  AchterbergJohn19710708  Achterberg     John  John Achterberg  1971-07-08   
2             AdamsArthur       Adams   Arthur     Arthur Adams         NaN   
3     AdamsEdward19081112       Adams   Edward     Edward Adams  1908-11-12   
4     AdkinsNigel19650311      Adkins    Nigel     Nigel Adkins  1965-03-11   

    comp_rec_pos soccerbase_pos    tm_pos_1 tm_pos_2 tm_pos_3  
0   Outside Left            NaN         NaN      NaN      NaN  
1            NaN     Goalkeeper  Goalkeeper      NaN      NaN  
2  Outside Right            NaN         NaN      NaN      NaN  
3  Outside Right            NaN         NaN      NaN      NaN  
4     Goalkeeper            NaN  Goalkeeper      NaN      NaN

In [6]:
duplicates = all_players['player_id'].value_counts().to_frame().rename(columns={'index': 'player_id', 'player_id': 'count'}).reset_index().query('count > 1')['player_id'].to_list()

all_players.query("player_id.isin(@duplicates)")

Empty DataFrame
Columns: [player_id, surname, forename, player_name, player_dob, comp_rec_pos, soccerbase_pos, tm_pos_1, tm_pos_2, tm_pos_3]
Index: []

In [7]:
all_players.to_csv("./output/player-positions.csv", index=False)